## Task 1: Histogram of Gradients (HoG)
-----------------------------------------------------------------------------------
Author: Rajesh Siraskar
Created: 01-Jan-2019
- 01-Jan-2019: First version- Create skeleton for function
- 02-Jan-2019: Test grads and magnitude/angles with a test image
    - Pre-processing. Add bi-lateral filtering to preserve edges
- 03-Jan-2019: Step.3.a.: Scan image and gather Im and Ia in num_bins
- 04-Jan-2019: Step.3.a.: Understanding what to bin! Created code for creating 20 deg. bins
    - Study the orginal thesis by Dalal. Understand recommendations
    - Developing binning logic. Cell based Im Ia calculated
    - Binned using naive logic. If Ia is within bin, then Im goes into that bin. Issue: Need to be lists of Im? Right now Im is over-writing
    - 5 for-loops for the binning. Need to use list-comprehension if logic correct
- 05-Jan-2019: Looked at the original CPP code, difficult to understand - however the **original .cpp** version shows 4 for loops!
    - It is confirmed that the pixel level loop is required. Approach looks good so far.
- 06-Jan-2019: Need to HOG_LAB.py to really understand Dalal N., several concepts solidified
    - Move from lab to main notebook
- 06-Jan-2019: Non vectorized code: Test images: 8, 4.86MB: Persp. map time: 13.3 mins.  
- 10-Jan-2019: Cleaned version


**References:**
- Algorithm: Klette 'Concise Computer Vision'. Pg.382
- Original source: Ph.D. Thesis, Navneet DALAL, 2006: 'Finding People in Images and Videos'
- Explanation: Satya Mallick, Big Vision LLC, 2018
- Dec. 2016, https://www.learnopencv.com/histogram-of-oriented-gradients/
- (Mordvintsev A. and Abid K., 2014), cv2.bilateralFilter() is highly effective at noise removal while still preserving edges at the cost of the operation being slower compared to other filters
- https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html

In [8]:
# Jupyter specific command to plot images inline with document
%matplotlib inline

# Import modules
import cv2
from matplotlib import pyplot as plt # for displaying images and plots
import numpy as np

## Function to compute histogram of gradients for an image

Main function applies the following steps:
    
**Step 1.: Pre-processing**
    [a] Intensity normalization: Simple normalization applied w.r.t. max intensity 255
    [b] Smoothing filter: **Bilateral Filtering** applied
    
The algorithm from the Klette 'Concise Computer Vision'. Pg.382

The original thesis (Dalal N., 2006: pg. 37 - 4.3.2 Gradient Computation) suggests recommends sigma=0 i.e. _NO_ smoothing

However tests showed that the edges were more prominient with use of the bi-lateral filter and might prove more effective for pedestrian detection. The student-author therefore proposes and has implemented **bilateral filtering** instead. The application of course can be controlled using the parameter 

As stated in OpenCV-Python Tutorials (Mordvintsev A. and Abid K., 2014), cv2.bilateralFilter() is highly effective at noise removal while still preserving edges at the cost of the operation being slower compared to other filters

- Ref.: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html
- Recommended values ref https://docs.opencv.org/3.1.0/d4/d86/group__imgproc__filter.html#ga9d7064d478c95d60003cf839430737ed

**Step 2.a.: Calculate gradients for each pixel**

The Sobel filter with a kernel size 1 pixel x 1 pixel is used.
- Ref.: (Dalal N., 2006: pg. 37 - 4.3.2 Gradient Computation) suggests Sobel operator [−1, 0, 1]

Also absolute values of gradients is used   
- Use un-signed gradients pg. 48. '4.6 Experiments on Other Classes'

**Step 2.b.: Compute the magnitude and angles of the gradient vectors**
- Magnitude and angle of a 2D vector can be computed using Polar conversion  

**Step.3: Spatial binning**
- Group pixels in non-overlapping cells (e.g. 8×8 pixels).
- See function _HoG_histogram_binning() below. The outline of algorithm based on original thesis (Dalal N., 2006: Appendix D) 

Final step is the loop that traverses the image using blocks of cells and moving one block-stride at a time, starting at the top-left corner and scanning.

### Known issue:
- The student has attempted an implementation by studying the algorithm provided (Klette, R. 2014) and orginal thesis (Dalal, N. 2006)
- Although the student has used list comprehension to eliminate 3 inner loops from the starting implementation of 5 for-loops, the implementation has still turned out to be **extremely slow**
- Approx. time to scan one test image: **48.05 secs**

**Parameters:**

- Cells: A unit block of pixels
      cell_size = size of cell in nxn pixels  (default = 8x8)
- Block cells: n cells
      block_cells = multiplier n i.e. n cells HxW  (default = 2)
- Block step: Cells by which steps are made - scanning bounding box left to right and top to bottom
      step_cells: n cells (default = 1)        
- Number of 'directional' (bins of direction i.e. angle of the gradient vectors) 20 deg. bins => 180 degs./20 = 9 bins
      num_bins (default = 9)
- Apply smoothing filter?
      apply_filter (default = None)
- If filter applied then supply an additional parameter
      filter_param (default = None)


In [9]:
## Function to compute histogram of gradients for an image

def safe_HoG_V2 (image, cell_size = (8, 8), block_cells = 2, step_cells = 1, num_bins = 9, apply_filter = None, filter_param = None):

    # Resize image to recommended size 64×128 [(Dalal N., 2006) Table 4.2. Key HOG parameters :pg 47]
    
    # Size of bounding box     
    image_size = (image.shape[0], image.shape[1])
    
    # Warning: cv2.resize in cols x rows 
    recommended_size = (64, 128)
    image = cv2.resize(image, recommended_size)
    
    # Block size nxn cells. n = block_cells
    block_size =  (block_cells*cell_size[0], block_cells*cell_size[1])

    # Block stride - can move say one cell at a time, scanning left to right and top to bottom
    block_stride = step_cells*cell_size[0]

    ### Compute Histogram of Gradients
    # Algorithm: Klette 'Concise Computer Vision'. Pg.382
    # Refinements based on the original thesis (Dalal N., 2006)
    
    # Step 1.: Pre-processing
    #          [a] Intensity normalization
    #          [b] Smoothing filter:
    
    # Step 1.a: Normalize with highest value of intensity
    #    Simplest form of normalization used
    #    Thesis suggests 'square root gamma compression'
    
    image = np.float32(image)/255.0
    
    # Step 1.b: Bilateral Filtering
    #    Normal smoothing including Gaussian blur do remove noise but affect the edges too.
    #    For pedestrian detection it might be beneficial to preserve the edges - hence we use a bilateral filter instead
    #    As stated in OpenCV-Python Tutorials (Mordvintsev A. and Abid K., 2014), cv2.bilateralFilter() is highly effective 
    #      at noise removal while still preserving edges at the cost of the operation being slower compared to other filters
    #      https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html
    #    Recommended values ref https://docs.opencv.org/3.1.0/d4/d86/group__imgproc__filter.html#ga9d7064d478c95d60003cf839430737ed
    # 
    #    The original thesis (Dalal N., 2006: pg. 37 - 4.3.2 Gradient Computation) suggests
    #      recommends sigma=0 i.e. NO smoothing
    #
    #    However tests showed that the edges were more prominient with use of the bi-lateral filter
    
    if (apply_filter == 'GAUSSIAN'): 
        if (filter_param is None): filter_param = 5
        # All three parameters (ksize.width, ksize.height) and sigma assumed to take the value filter_param
        image = cv2.GaussianBlur(image, (filter_param, filter_param), filter_param)
        
    if (apply_filter == 'BILATERAL'): 
        if (filter_param is None): filter_param = 80
        image = cv2.bilateralFilter(image, 7, filter_param, filter_param)
    
    # Step 2.a.: Calculate gradients for each pixel
    #   Use Sobel filter with a kernel size 1 pixel x 1 pixel
    #   Ref.: (Dalal N., 2006: pg. 37 - 4.3.2 Gradient Computation) suggests Sobel operator [−1, 0, 1]
    #   Use un-signed gradients pg. 48. '4.6 Experiments on Other Classes'
    
    grad_x = abs(cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=1))
    grad_y = abs(cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=1))


    # Step 2.b.: Compute the magnitude and angles of the gradient vectors 
    Im, Ia = cv2.cartToPolar(grad_x, grad_y, angleInDegrees=True)
    

    # Step.3: Spatial binning
    # Group pixels in non-overlapping cells (e.g. 8×8 pixels)
    # Use maps Im and Ia to accumulate magnitude values into direction bins 
    #  (e.g., nine bins for intervals of 20◦ each, for covering a full 180◦ range) 
    #  to obtain a voting vector (e.g. of length 9) for each cell
    
    # 1 histogram per cell with 9 bins
    # Create a collection of bins of num_bins length
    rows = int(image.shape[0]/cell_size[0])
    cols = int(image.shape[1]/cell_size[1])
    Im_bins =  np.zeros((rows, cols, num_bins))
    
    # Step 3.a.: Create bin centers (e.g. 0, 20, 40...160 deg., if num_bins=9)
    angle_slices = int(180.0/num_bins)
    Ia_bin_centers = [angle for angle in range(0, 180, angle_slices)]

 
    ## Traverse the image starting at top, then left to right, then next row...

    # Keep track of cells
    n_cell = 0
    n_row = 0
    n_col = 0

    # Histogram bin band-width
    b = Ia_bin_centers[1] - Ia_bin_centers[0]

    # Move T-B
    n_row = 0
    for y in range(0, image.shape[0], block_stride):
        # Move L-R, intialize column counter
        n_col = 0
        for x in range(0, image.shape[1], block_stride):
            Im_bins = _HoG_histogram_binning (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, x, y, cell_size, num_bins)
            # Complete cell covered, move to next cell 
            n_col += 1
        # 1 row of cells L-R covered, proceed to next row
        n_row+=1
    # All rows covered L-R, complete image processed

    # Create a feature vector
    hog_features = Im_bins.ravel().reshape(-1, 1)
                    
    return (grad_x, grad_y, Im, hog_features)  

In [10]:
## Function to compute histogram of gradients for an image

def HoG_V2 (image, cell_size = (8, 8), block_cells = 2, step_cells = 1, num_bins = 9, apply_filter = None, filter_param = None):

    # Resize image to recommended size 64×128 [(Dalal N., 2006) Table 4.2. Key HOG parameters :pg 47]
    
    # Size of bounding box     
    image_size = (image.shape[0], image.shape[1])
    
    # Warning: cv2.resize in cols x rows 
    recommended_size = (64, 128)
    image = cv2.resize(image, recommended_size)
    
    # Block size nxn cells. n = block_cells
    block_size =  (block_cells*cell_size[0], block_cells*cell_size[1])

    # Block stride - can move say one cell at a time, scanning left to right and top to bottom
    block_stride = step_cells*cell_size[0]

    ### Compute Histogram of Gradients
    # Algorithm: Klette 'Concise Computer Vision'. Pg.382
    # Refinements based on the original thesis (Dalal N., 2006)
    
    # Step 1.: Pre-processing
    #          [a] Intensity normalization
    #          [b] Smoothing filter:
    
    # Step 1.a: Normalize with highest value of intensity
    #    Simplest form of normalization used
    #    Thesis suggests 'square root gamma compression'
    
    image = np.float32(image)/255.0
    
    # Step 1.b: Bilateral Filtering
    #    Normal smoothing including Gaussian blur do remove noise but affect the edges too.
    #    For pedestrian detection it might be beneficial to preserve the edges - hence we use a bilateral filter instead
    #    As stated in OpenCV-Python Tutorials (Mordvintsev A. and Abid K., 2014), cv2.bilateralFilter() is highly effective 
    #      at noise removal while still preserving edges at the cost of the operation being slower compared to other filters
    #      https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html
    #    Recommended values ref https://docs.opencv.org/3.1.0/d4/d86/group__imgproc__filter.html#ga9d7064d478c95d60003cf839430737ed
    # 
    #    The original thesis (Dalal N., 2006: pg. 37 - 4.3.2 Gradient Computation) suggests
    #      recommends sigma=0 i.e. NO smoothing
    #
    #    However tests showed that the edges were more prominient with use of the bi-lateral filter
    
    if (apply_filter == 'GAUSSIAN'): 
        if (filter_param is None): filter_param = 5
        # All three parameters (ksize.width, ksize.height) and sigma assumed to take the value filter_param
        image = cv2.GaussianBlur(image, (filter_param, filter_param), filter_param)
        
    if (apply_filter == 'BILATERAL'): 
        if (filter_param is None): filter_param = 80
        image = cv2.bilateralFilter(image, 7, filter_param, filter_param)
    
    # Step 2.a.: Calculate gradients for each pixel
    #   Use Sobel filter with a kernel size 1 pixel x 1 pixel
    #   Ref.: (Dalal N., 2006: pg. 37 - 4.3.2 Gradient Computation) suggests Sobel operator [−1, 0, 1]
    #   Use un-signed gradients pg. 48. '4.6 Experiments on Other Classes'
    
    grad_x = abs(cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=1))
    grad_y = abs(cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=1))


    # Step 2.b.: Compute the magnitude and angles of the gradient vectors 
    Im, Ia = cv2.cartToPolar(grad_x, grad_y, angleInDegrees=True)
    

    # Step.3: Spatial binning
    # Group pixels in non-overlapping cells (e.g. 8×8 pixels)
    # Use maps Im and Ia to accumulate magnitude values into direction bins 
    #  (e.g., nine bins for intervals of 20◦ each, for covering a full 180◦ range) 
    #  to obtain a voting vector (e.g. of length 9) for each cell
    
    # 1 histogram per cell with 9 bins
    # Create a collection of bins of num_bins length
    rows = int(image.shape[0]/cell_size[0])
    cols = int(image.shape[1]/cell_size[1])
    Im_bins =  np.zeros((rows, cols, num_bins))
    
    # Step 3.a.: Create bin centers (e.g. 0, 20, 40...160 deg., if num_bins=9)
    angle_slices = int(180.0/num_bins)
    Ia_bin_centers = [angle for angle in range(0, 180, angle_slices)]

 
    ## Traverse the image starting at top, then left to right, then next row...

    # Keep track of cells
    n_cell = 0
    n_row = 0
    n_col = 0

    # Histogram bin band-width
    b = Ia_bin_centers[1] - Ia_bin_centers[0]

    n_cols = range(0, image.shape[1], block_stride)
    
    
    # Move T-B
    n_row = 0
    for y in range(0, image.shape[0], block_stride):
        # Move L-R, intialize column counter
        n_col = 0        
        
        Im_bins = [_HoG_histogram_binning (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, x, y, cell_size, num_bins) 
                          for n_col in enumerate(n_cols)]
        
        #for x in range(0, image.shape[1], block_stride):
        #    Im_bins = _HoG_histogram_binning (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, x, y, cell_size, num_bins)
        #    # Complete cell covered, move to next cell 
        #    n_col += 1
        # 1 row of cells L-R covered, proceed to next row
        n_row+=1
    # All rows covered L-R, complete image processed

    # Create a feature vector
    hog_features = Im_bins.ravel().reshape(-1, 1)
                    
    return (grad_x, grad_y, Im, hog_features)  

In [11]:
def _HoG_process_roi_across (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, x, y, cell_size, num_bins):
    
    [_HoG_process_pixel(Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col,  x, y, px_row, px_col, num_bins) 
     for px_row in range(y, y+cell_size[1]) for px_col in range(x, x+cell_size[0])]

    n_col += 1
    return (Im_bins, n_col)

### Function: _HoG_histogram_binning

#### HoG Vector based on thesis (Dalal N., 2006: Appendix D)

        Here the bins are based on 9 centers:
        Ia_bins:  [0, 20, 40, 60, 80, 100, 120, 140, 160]
        
        Ph.D. Thesis, Navneet DALAL, 2006: 'Finding People in Images and Videos', Appendix 'D'
        
        If x is value to be interpolated into bins and x is such that it lies between x1 and x2,
        where x1 and x2 are centers of two bins and b is the band-width of the bins:
        
        x1 <= x < x2
        
        h(x1) <- h(x1) + w * [1 - (x-x1)/b]
        h(x2) <- h(x1) + w * (x-x1)/b
        
        **Note**: The thesis pg: 131, Appendix D, could there be an error? h(x1) mentioned in the eqn. for h(x2)?
        
        Example:
        If Im = 32.00, the bin centers are 20 <= x < 40
        h(x1) <- h(x1) + w * [1 - (32-20)/20] = h(x1) + w * 0.9
        h(x2) <- h(x2) + w * [(32-20)/20] = h(x2) + w * 0.1

In [12]:
### Access each pixel in the cell and call _HoG_process_pixel for each pixel
    ### Binning logic: Dalal N., 2006: Appendix D
    # Place the pixel magnitude Im in the appropriate bin
    # x1 <= x < x2
    # h(x1) <- h(x1) + w * [1 - (x-x1)/b]
    # h(x2) <- h(x1) + w * (x-x1)/b
    
    # For this implementation it is assumed that w = Im at that pixel

def _HoG_histogram_binning (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, x, y, cell_size, num_bins):
        
    [_HoG_process_pixel(Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col,  x, y, px_row, px_col, num_bins) 
     for px_row in range(y, y+cell_size[1]) for px_col in range(x, x+cell_size[0])]

    n_col += 1

    return (Im_bins, n_col)

### Function: _HoG_process_pixel

Process each pixel and if the Ia is within the bin-centers' range then call  _HoG_process_bin 

In [13]:
### Process each pixel and if the Ia is within the bin-centers' range then call  _HoG_process_bin 
def _HoG_process_pixel (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col,  x, y, px_row, px_col, num_bins):
    
    
    [_HoG_process_bin (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, px_row, px_col, n_bin, num_bins) 
     if (Ia[px_row, px_col] >= Ia_bin_centers[n_bin] and Ia[px_row, px_col] < Ia_bin_centers[n_bin+1]) else None 
     for n_bin in range(9)] 

### Function: _HoG_process_bin

Process Im and bin it. This function handles the logic of splitting the 'vote' if the bin is the last one and sharing it with the first bin 

In [14]:
### Process Im and bin it
def _HoG_process_bin (Im, Ia, Im_bins, Ia_bin_centers, b, n_row, n_col, px_row, px_col, n_bin, num_bins):
    
    if (n_bin > -1):
        # Allow if we are at last-but-one bin, but if last bin then split share with 0 deg. (1st) bin:
        next_bin = (n_bin+1) if (n_bin+1 < num_bins) else 0

        Im_bins[n_row][n_col][n_bin] += Im[px_row,px_col] * (1 - (Ia[px_row,px_col] - Ia_bin_centers[n_bin])/b)
        Im_bins[n_row][n_col][next_bin] += Im[px_row,px_col] * (Ia[px_row,px_col] - Ia_bin_centers[n_bin])/b
    
    return(Im_bins)

### Test the HoG Vector

Read a sample image and test the HoG vector output

In [15]:
# Read image
img_bgr  = cv2.imread('images/Lab/HoG_lab_2_pedestrian.PNG')
img_rgb  = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)  
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

In [16]:
%%time
(grad_x, grad_y, Im, hog_features) = HoG_V2(img_gray)
fig, axes = plt.subplots(1, 5, figsize=(8, 8), gridspec_kw=dict(hspace=0.1, wspace=0.1))

axes[0].imshow(img_rgb)
axes[0].set(xticks=[], yticks=[], xlabel='org.')

axes[1].imshow(img_gray, cmap='bone')
axes[1].set(xticks=[], yticks=[], xlabel='gray img.')

axes[2].imshow(grad_x, cmap='bone')
axes[2].set(xticks=[], yticks=[], xlabel='grad.x')
     
axes[3].imshow(grad_y, cmap='bone')
axes[3].set(xticks=[], yticks=[], xlabel='grad.y')

axes[4].imshow(Im, cmap='bone')
axes[4].set(xticks=[], yticks=[], xlabel='magnitudes')

NameError: name 'x' is not defined

In [11]:
## Approximate time estimation:
    
# Time required to run HoG on a single ROI
roi_size = (64, 128)
t = 0.383

# For testing it was determined that the size for reasonable performance was
(test_image_H, test_image_W) = (400, 500)

# View-port sizing
(viewport_H, viewport_W) =  (216, 82)

# View-port stride as it scans the test image
STEP_SIZE_FACTOR = 0.30

bounding_boxes_across = test_image_W/(STEP_SIZE_FACTOR*viewport_W)
bounding_boxes_down = test_image_H/(STEP_SIZE_FACTOR*viewport_H)

total_bbs = bounding_boxes_across*bounding_boxes_down

total_time = t*total_bbs

print('\n Approx. time to scan one test image: {:4.2f} secs'.format(total_time))
print(' Approx. time for {} test images {:4.2f} mins'.format(26, 26*total_time/60))


 Approx. time to scan one test image: 48.05 secs
 Approx. time for 26 test images 20.82 mins
